In [14]:
# load tydi json and compute stats on the questions and answers
import glob
import gzip
import json
import spacy

nlp = spacy.load('en_core_web_sm')

train_data_dir = "/dccstor/srosent2/primeqa/data/nq-lfqa-train/*jsonl"
dev_data_dir = "/dccstor/srosent2/primeqa/data/dev/nq-lfqa/*jsonl"

train_files = glob.glob(train_data_dir)
dev_files = glob.glob(dev_data_dir)

In [2]:

def load_json_from_file(gt_file_patterns):
    data = []
    if gt_file_patterns.endswith('gz'):
        f = gzip.open(gt_file_patterns, 'rt', encoding='utf-8')
    else:
        f = open(gt_file_patterns, 'rt', encoding='utf-8')
    lines = f.readlines()
    for line in lines:
        data.append(json.loads(line))
    return data


In [ ]:
from random import sample

def compute_stats(data, type):
    first_word_stat = {}
    for example in data:
        qtype = example['type'][0]
        
        if qtype != type:
            continue
        example_id = example['example_id']
        question = example["question_text"]

        q_word = question.split()[0]

        annotation = example['annotations'][0]

        if annotation == None:
            continue

        minimal_text = ""
        if annotation['minimal_answer']['plaintext_start_byte'] != -1:
            minimal_text = example['document_plaintext'].encode('utf-8')[annotation['minimal_answer']['plaintext_start_byte']:annotation['minimal_answer']['plaintext_end_byte']].decode('utf-8')
        passage_offsets = example['passage_answer_candidates'][annotation['passage_answer']['candidate_index']]
        passage_text = example['document_plaintext'].encode('utf-8')[passage_offsets['plaintext_start_byte']:passage_offsets['plaintext_end_byte']].decode('utf-8')
        info = {}
        info["question"] = question
        info["passage_text"] = passage_text
        info["minimal_text"] = minimal_text
        info["example_id"] = example_id
        if q_word in first_word_stat:
            first_word_stat[q_word].append(info)
        else:
            first_word_stat[q_word] = []
            first_word_stat[q_word].append(info)
    for word in first_word_stat:
        if len(first_word_stat[word]) > 20:
            sample_questions = sample(first_word_stat[word],5)
            print(word + ": " + str(len(first_word_stat[word])))
            for sample_question in sample_questions:
                print(str(sample_question['example_id']) + "\t" + sample_question['question'] + "\t" + sample_question['passage_text'] + "\t" + sample_question['minimal_text'])
                    

In [3]:
dev_data = []
for dev_file in dev_files:
    dev_data.extend(load_json_from_file(dev_file))

#compute_stats(dev_data, "la")

In [15]:
train_data = []
for train_file in train_files:
    train_data.extend(load_json_from_file(train_file))

In [ ]:
compute_stats(train_data, "la")

In [22]:
def get_appen_data(data, type):
    count = 0
    for example in data:
        qtype = example['type'][0]
        
        if qtype != type:
            continue
        example_id = example['example_id']
        question = example["question_text"]

        q_word = question.split()[0]

        annotation = example['annotations'][0]

        if annotation == None:
            continue

        minimal_text = ""
        if annotation['minimal_answer']['plaintext_start_byte'] != -1:
            minimal_text = example['document_plaintext'].encode('utf-8')[annotation['minimal_answer']['plaintext_start_byte']:annotation['minimal_answer']['plaintext_end_byte']].decode('utf-8')
        passage_offsets = example['passage_answer_candidates'][annotation['passage_answer']['candidate_index']]
        passage_text = example['document_plaintext'].encode('utf-8')[passage_offsets['plaintext_start_byte']:passage_offsets['plaintext_end_byte']].decode('utf-8')
        
        passage_sentences = ""
        for sentence in nlp(passage_text).sents:
            passage_sentences += sentence.text + "&nbsp;"

        count += 1
        if count < 100:
            continue
        if count > 200:
            break

        print(str(example_id) + "\t" + question + "\t" + example['document_title'] + "\t" + example['document_url'] + "\t" + passage_sentences + "\t" + minimal_text)
             

In [23]:
get_appen_data(train_data, "la")

-3077016148542864018	who played the guitar solo in while my guitar gently weeps	While My Guitar Gently Weeps	https://en.wikipedia.org//w/index.php?title=While_My_Guitar_Gently_Weeps&amp;oldid=852730329	The Beatles recorded `` While My Guitar Gently Weeps '' several times during the sessions for their self - titled double album , also known as `` the White Album '' .&nbsp;The recording sessions , which began in late May 1968 , were characterised by a lack of cooperation among the four band members , and by what Lennon 's bandmates regarded as the overly intrusive presence of his new romantic partner , Yoko Ono .&nbsp;In this atmosphere , Harrison had initially been reluctant to present his new compositions to the group .&nbsp;Take 1 on 25 July -- the version later issued on Anthology 3 -- was a solo performance by Harrison , playing his Gibson J - 200 acoustic guitar , with an overdubbed harmonium part .&nbsp;	
-2021144360257334672	who won the wwe money in the bank match	Money in the Ba